In [1]:
import pandas as pd
import pickle
import statistics
from statistics import mean
from sklearn import metrics
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

In [3]:
with open('df_train.pkl', 'rb') as f:
    df_train = pickle.load(f)
df_train.shape

(69445, 2423)

In [4]:
with open('df_validation.pkl', 'rb') as f:
    df_validation = pickle.load(f)
df_validation.shape

(22537, 2423)

In [5]:
with open('df_test.pkl', 'rb') as f:
    df_test = pickle.load(f)
df_test.shape

(22289, 2423)

In [15]:
df_train_4 = df_train[[
'INTERVAL_IN_FILE',
'BAND_ARR_5',
'BAND_ARR_6',
'BAND_ARR_7',
'BAND_ARR_8',
'BAND_ARR_13',
'BIT_FLAG',
'MAX_VALUE',
'NTC',
'BAND_ARR_2__fft_coefficient__attr_"imag"__coeff_1',
'BAND_ARR_8__fft_coefficient__attr_"real"__coeff_4',
'VALUE'

]].reset_index()

df_validation_4 = df_validation[[
'INTERVAL_IN_FILE',
'BAND_ARR_5',
'BAND_ARR_6',
'BAND_ARR_7',
'BAND_ARR_8',
'BAND_ARR_13',
'BIT_FLAG',
'MAX_VALUE',
'NTC',
'BAND_ARR_2__fft_coefficient__attr_"imag"__coeff_1',
'BAND_ARR_8__fft_coefficient__attr_"real"__coeff_4',
'VALUE'

]].reset_index()

df_test_4 = df_test[[
'INTERVAL_IN_FILE',
'BAND_ARR_5',
'BAND_ARR_6',
'BAND_ARR_7',
'BAND_ARR_8',
'BAND_ARR_13',
'BIT_FLAG',
'MAX_VALUE',
'NTC',
'BAND_ARR_2__fft_coefficient__attr_"imag"__coeff_1',
'BAND_ARR_8__fft_coefficient__attr_"real"__coeff_4',
'VALUE'

]].reset_index()



In [7]:
df_for_model_tab = pd.DataFrame(columns = ['validation metric'])
df_for_model_tab['validation metric'] = ('F1', 'Accuracy', 'Precision', 'Recall', 'max difference from error mean', 'mean difference from error mean', 'number of bigger than 0.2 differnece', 'number of bigger than 0.1 differnece')
df_for_model_tab

,validation metric
0,F1
1,Accuracy
2,Precision
3,Recall
4,max difference from error mean
5,mean difference from error mean
6,number of bigger than 0.2 differnece
7,number of bigger than 0.1 differnece


In [18]:
def error_per_session(train_data, test_data, model, model_name):

    X_train = train_data.loc[:, train_data.columns != 'VALUE']
    X_test = test_data.loc[:, test_data.columns != 'VALUE']
    number = preprocessing.LabelEncoder()
    X_train['SESSION_INFO_ID'] = number.fit_transform(X_train.SESSION_INFO_ID)       
    X_test['SESSION_INFO_ID'] = number.fit_transform(X_test.SESSION_INFO_ID)       

    y_train = train_data['VALUE']
    y_test = test_data['VALUE']

    
    clf = model
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    prediction_row_check = pd.DataFrame(columns = ['y pred', 'y'])
    prediction_row_check['y']=y_test
    prediction_row_check['y pred']=y_pred
    prediction_row_check['SESSION_INFO_ID']=X_test['SESSION_INFO_ID']
    prediction_row_check['is prediction wrong'] = (prediction_row_check['y pred'] - prediction_row_check['y'])**2
    
    prediction_session_check=pd.DataFrame(prediction_row_check.groupby('SESSION_INFO_ID')['is prediction wrong'].mean())
    prediction_session_check = prediction_session_check.reset_index()
    
    prediction_session_check['abs difference from average'] = abs(prediction_session_check['is prediction wrong']-statistics.mean(prediction_row_check['is prediction wrong']))
    
    f1 = metrics.f1_score(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    max_difference_from_error_mean = prediction_session_check['abs difference from average'].max()
    mean_difference_from_error_mean = prediction_session_check['abs difference from average'].mean()
    number_of_bigger_than_02_differnece = prediction_session_check[prediction_session_check['abs difference from average']>0.2]['SESSION_INFO_ID'].count()
    number_of_bigger_than_01_differnece = prediction_session_check[prediction_session_check['abs difference from average']>0.1]['SESSION_INFO_ID'].count()
    df_for_model_tab[model_name] = f1, accuracy, precision, recall, max_difference_from_error_mean , mean_difference_from_error_mean, number_of_bigger_than_02_differnece, number_of_bigger_than_01_differnece



In [19]:
error_per_session(df_train_4, df_validation_4, RandomForestClassifier(), 'Random Forest')
df_for_model_tab

C:\Users\Arie\AppData\Local\Temp/ipykernel_5564/2063655791.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['SESSION_INFO_ID'] = number.fit_transform(X_train.SESSION_INFO_ID)
C:\Users\Arie\AppData\Local\Temp/ipykernel_5564/2063655791.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['SESSION_INFO_ID'] = number.fit_transform(X_test.SESSION_INFO_ID)


,validation metric,Random Forest
0,F1,0.977957
1,Accuracy,0.981852
2,Precision,0.980759
3,Recall,0.975172
4,max difference from error mean,0.175882
5,mean difference from error mean,0.022414
6,number of bigger than 0.2 differnece,0.000000
7,number of bigger than 0.1 differnece,1.000000


In [20]:
error_per_session(df_train_4, df_validation_4, GradientBoostingClassifier(), 'Gradient Boost')
df_for_model_tab

C:\Users\Arie\AppData\Local\Temp/ipykernel_5564/2063655791.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['SESSION_INFO_ID'] = number.fit_transform(X_train.SESSION_INFO_ID)
C:\Users\Arie\AppData\Local\Temp/ipykernel_5564/2063655791.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['SESSION_INFO_ID'] = number.fit_transform(X_test.SESSION_INFO_ID)


,validation metric,Random Forest,Gradient Boost
0,F1,0.977957,0.967270
1,Accuracy,0.981852,0.973200
2,Precision,0.980759,0.975410
3,Recall,0.975172,0.959265
4,max difference from error mean,0.175882,0.184287
5,mean difference from error mean,0.022414,0.031394
6,number of bigger than 0.2 differnece,0.000000,0.000000
7,number of bigger than 0.1 differnece,1.000000,2.000000


In [21]:
error_per_session(df_train_4, df_validation_4, AdaBoostClassifier(), 'Ada Boost')
df_for_model_tab

C:\Users\Arie\AppData\Local\Temp/ipykernel_5564/2063655791.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['SESSION_INFO_ID'] = number.fit_transform(X_train.SESSION_INFO_ID)
C:\Users\Arie\AppData\Local\Temp/ipykernel_5564/2063655791.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['SESSION_INFO_ID'] = number.fit_transform(X_test.SESSION_INFO_ID)


,validation metric,Random Forest,Gradient Boost,Ada Boost
0,F1,0.977957,0.967270,0.946152
1,Accuracy,0.981852,0.973200,0.957004
2,Precision,0.980759,0.975410,0.979519
3,Recall,0.975172,0.959265,0.914983
4,max difference from error mean,0.175882,0.184287,0.255174
5,mean difference from error mean,0.022414,0.031394,0.046640
6,number of bigger than 0.2 differnece,0.000000,0.000000,1.000000
7,number of bigger than 0.1 differnece,1.000000,2.000000,2.000000


In [22]:
from sklearn.naive_bayes import GaussianNB

In [23]:
error_per_session(df_train_4, df_validation_4, GaussianNB(), 'Gaussian Naive Bayes')
df_for_model_tab

C:\Users\Arie\AppData\Local\Temp/ipykernel_5564/2063655791.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['SESSION_INFO_ID'] = number.fit_transform(X_train.SESSION_INFO_ID)
C:\Users\Arie\AppData\Local\Temp/ipykernel_5564/2063655791.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['SESSION_INFO_ID'] = number.fit_transform(X_test.SESSION_INFO_ID)


,validation metric,Random Forest,Gradient Boost,Ada Boost,Gaussian Naive Bayes
0,F1,0.977957,0.967270,0.946152,0.817446
1,Accuracy,0.981852,0.973200,0.957004,0.864978
2,Precision,0.980759,0.975410,0.979519,0.925051
3,Recall,0.975172,0.959265,0.914983,0.732266
4,max difference from error mean,0.175882,0.184287,0.255174,0.644843
5,mean difference from error mean,0.022414,0.031394,0.046640,0.123599
6,number of bigger than 0.2 differnece,0.000000,0.000000,1.000000,2.000000
7,number of bigger than 0.1 differnece,1.000000,2.000000,2.000000,43.000000


In [24]:
from sklearn.neighbors import KNeighborsClassifier

In [25]:
error_per_session(df_train_4, df_validation_4, KNeighborsClassifier(), 'KNN')
df_for_model_tab

C:\Users\Arie\AppData\Local\Temp/ipykernel_5564/2063655791.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['SESSION_INFO_ID'] = number.fit_transform(X_train.SESSION_INFO_ID)
C:\Users\Arie\AppData\Local\Temp/ipykernel_5564/2063655791.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['SESSION_INFO_ID'] = number.fit_transform(X_test.SESSION_INFO_ID)


,validation metric,Random Forest,Gradient Boost,Ada Boost,Gaussian Naive Bayes,KNN
0,F1,0.977957,0.967270,0.946152,0.817446,0.855209
1,Accuracy,0.981852,0.973200,0.957004,0.864978,0.894662
2,Precision,0.980759,0.975410,0.979519,0.925051,0.988579
3,Recall,0.975172,0.959265,0.914983,0.732266,0.753547
4,max difference from error mean,0.175882,0.184287,0.255174,0.644843,0.655259
5,mean difference from error mean,0.022414,0.031394,0.046640,0.123599,0.117612
6,number of bigger than 0.2 differnece,0.000000,0.000000,1.000000,2.000000,2.000000
7,number of bigger than 0.1 differnece,1.000000,2.000000,2.000000,43.000000,45.000000
